In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential

2022-07-19 11:53:50.737611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-19 11:53:50.737656: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
############
# Functions
############

# Our 3d parameter space is the surface of a sphere centered at (1, 1, 1), with radius 1/sqrt(2)
# x, y coordinates are the x and y variances of our 2d Gaussian
# z coordinate determines the x-coordinate of the mean, mu = (z, 0)

def mean_gen(theta, phi):
    mu = 0.5 + (np.sqrt(2)/2) * np.cos(phi + np.pi / 4)
    return mu


def varx_gen(theta, phi):
    vx = 1.5 - (np.sqrt(2)/2) * np.cos(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vx


def vary_gen(theta, phi):
    vy = 1.5 - (np.sqrt(2)/2) * np.sin(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vy


# Generate n data
def spherical_data(n, thetas, phis, rand=1234):
    
    mx1, my1 = np.zeros(n), np.zeros(n)
    vx1, vy1 = np.ones(n), np.ones(n)
    
    mx2, my2 = mean_gen(thetas, phis), np.zeros(n)
    vx2, vy2 = varx_gen(thetas, phis), vary_gen(thetas, phis)
    
    x1, y1 = np.transpose(np.array([np.random.normal(mx1, vx1, size=n), 
                                    np.random.normal(my1, vy1, size=n), thetas, phis])), np.zeros(n)
    x2, y2 = np.transpose(np.array([np.random.normal(mx2, vx2, size=n), 
                                    np.random.normal(my2, vy2, size=n), thetas, phis])), np.ones(n)
    
    x, y = np.append(x1, x2, axis=0), np.append(y1, y2, axis=0)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = rand)
    
    return x_train, x_test, y_train, y_test

In [3]:
# Single Point on Sphere

n = 40000
rand_n = 12345
theta, phi = 3, 1
thetas, phis = theta * np.ones(n), phi * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)

model = tf.keras.models.load_model('3dmodels/discrete_model_mth15_mph15')

2022-07-19 11:54:01.623650: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-19 11:54:01.623688: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-19 11:54:01.623718: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-matan): /proc/driver/nvidia/version does not exist
2022-07-19 11:54:01.624076: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
for l in model.layers:
    l.trainable=False

inputs_hold = tf.keras.Input(shape=(1,))
simple_model = Dense(2, use_bias = False)(inputs_hold)
model3 = Model(inputs = inputs_hold, outputs = simple_model)

inputs = tf.keras.Input(shape=(2,))
inputs2 = tf.keras.layers.concatenate([inputs, model3(tf.ones_like(inputs)[:,0:1])])
hidden_layer_1 = model(inputs2)
# model2 = Model(inputs = inputs, outputs = hidden_layer_1)
# model2.compile(loss='categorical_crossentropy', optimizer='Adam', )

# model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like (TFOpLambda)       (None, 2)            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 1)            0           tf.ones_like[0][0]               
__________________________________________________________________________________________________
model (Functional)              (None, 2)            2           tf.__operators__.getitem[0][0]   
____________________________________________________________________________________________

In [8]:
print(x_train[0:10])

[[ 1.52149137  1.52567849  3.          1.        ]
 [-2.9591584  -1.67226417  3.          1.        ]
 [ 1.82442252 -1.09983086  3.          1.        ]
 [ 0.85642174 -0.14127318  3.          1.        ]
 [ 0.54308911 -1.12605262  3.          1.        ]
 [-1.01183886 -0.7713892   3.          1.        ]
 [ 1.75136515 -1.21595493  3.          1.        ]
 [-0.99150384  0.33114008  3.          1.        ]
 [-1.35306203 -1.03144875  3.          1.        ]
 [ 0.29660997 -0.39271103  3.          1.        ]]


In [5]:
X_train = x_train[:, 0:2]

In [7]:
for i in range(10):
    model2.fit(X_train, y_train, epochs=1,batch_size=100)
    print(i,"Fitted result: ", model2.trainable_weights[:])

640/640 [==============================] - 1s 2ms/step - loss: 0.0000e+00
0 Fitted result:  [<tf.Variable 'dense/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[30.175468, 17.86851 ]], dtype=float32)>]
640/640 [==============================] - 1s 2ms/step - loss: 0.0000e+00
1 Fitted result:  [<tf.Variable 'dense/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[30.811605, 18.24613 ]], dtype=float32)>]
640/640 [==============================] - 1s 2ms/step - loss: 0.0000e+00
2 Fitted result:  [<tf.Variable 'dense/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[31.447666, 18.629076]], dtype=float32)>]
640/640 [==============================] - 1s 2ms/step - loss: 0.0000e+00
3 Fitted result:  [<tf.Variable 'dense/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[32.084503, 19.006474]], dtype=float32)>]
640/640 [==============================] - 1s 2ms/step - loss: 0.0000e+00
4 Fitted result:  [<tf.Variable 'dense/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[32.72099 , 